<h1><center><strong> X-Ray Body Parts Prediction </center></strong></h1> 

Import Libraries

In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cv2
import tensorflow as tf
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Dropout, Flatten, InputLayer
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.callbacks import TensorBoard
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MultiLabelBinarizer

2023-08-19 15:41:26.001407: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-08-19 15:41:26.012497: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-19 15:41:26.156178: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-19 15:41:26.157773: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-19 15:41:26.910902: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

In [ ]:
BASE_DIR = os.path.dirname(os.getcwd())
DATA_DIR = os.path.join(BASE_DIR, 'data')
IMAGES_DIR = os.path.join(DATA_DIR, 'images')

MIXED_IMAGES = os.path.join(IMAGES_DIR, 'mixed')
WRIST_IMAGES = os.path.join(IMAGES_DIR, 'wrist')
LUMBER_SPINE_IMAGES = os.path.join(IMAGES_DIR, 'lumber_spine')
PELVIS_IMAGES = os.path.join(IMAGES_DIR, 'pelvis')
CHEST_IMAGES = os.path.join(IMAGES_DIR, 'chest')
HAND_IMAGES = os.path.join(IMAGES_DIR, 'hand')

Load images and labels from multiple directories into one DF
Steps (load_df):
    - Mixed
        - load df from csv
        - drop extra columns other than SOPInstance and Target
        - Change SOPInstance coloumn to 'Image name'
    - Lumber_spine
        - listdir() for all images in the directory
        - make a df with two columns 'Image name' and 'Target'
        - populate 'Image name' from listdir() and 'Target' to list interger as 14
    - Wrist
        - listdir() for all images in the directory
        - make a df with two columns 'Image name' and 'Target'
        - populate 'Image name' from listdir() and 'Target' to list interger as 21
    - Finally: Merge (Mixed, Lumber_spine and Wrist)
        - Merge all three df on axis=1 'Image name'
        

In [3]:
# preparing dataframe from 'mixed' directory 
mixed_df = pd.read_csv(os.path.join(os.getcwd(), 'data', 'train_df.csv'))

mixed_df = mixed_df[['SOPInstanceUID', 'Target']]
mixed_df = mixed_df.rename(columns={'SOPInstanceUID':'Instance'})

mixed_df['Target'] = mixed_df['Target'].apply(lambda x: x.strip())
mixed_df['Target'] = mixed_df['Target'].apply(lambda x: x.split(' '))

mixed_df['Instance'] = mixed_df['Instance'].apply(lambda x: x + '-c.png')

mixed_df.head(8)

,Instance,Target
0,1.2.826.0.1.3680043.8.498.10025629581362719970...,[0]
1,1.2.826.0.1.3680043.8.498.10036150326276641158...,[15]
2,1.2.826.0.1.3680043.8.498.10038426859954986240...,[12]
3,1.2.826.0.1.3680043.8.498.10050991192143676483...,[14]
4,1.2.826.0.1.3680043.8.498.10053309524595490852...,[3]
5,1.2.826.0.1.3680043.8.498.10053755320637729867...,[3]
6,1.2.826.0.1.3680043.8.498.10062189329714053601...,[14]
7,1.2.826.0.1.3680043.8.498.10065930002825553435...,"[13, 20]"


In [4]:
# preparing dataframe from 'Hand' directory
hand_images = os.listdir(HAND_IMAGES)
hand_images = [image_name for image_name in hand_images if image_name.lower().endswith('.png')]
hand_target = [[int(9)] for image_name in hand_images]

hand_df = pd.DataFrame({'Instance': hand_images, 'Target': hand_target})
hand_df.head(2)

,Instance,Target
0,imgtr33705.png,[9]
1,imgtr33905.png,[9]


In [5]:
# preparing dataframe from 'Chest' directory
chest_images = os.listdir(CHEST_IMAGES)
chest_images = [image_name for image_name in chest_images if image_name.lower().endswith('.jpeg')]
chest_target = [[int(3)] for image_name in chest_images]

chest_df = pd.DataFrame({'Instance': chest_images, 'Target': chest_target})
chest_df.head(2)

,Instance,Target
0,IM-0595-0001.jpeg,[3]
1,NORMAL2-IM-0775-0001.jpeg,[3]


In [6]:
# preparing dataframe from 'Pelvis' directory
pelvis_images = os.listdir(PELVIS_IMAGES)
pelvis_images = [image_name for image_name in pelvis_images if image_name.lower().endswith('.png')]
pelvis_target = [[int(15)] for image_name in pelvis_images]

pelvis_df = pd.DataFrame({'Instance': pelvis_images, 'Target': pelvis_target})
pelvis_df.head(2)

,Instance,Target
0,pelvicfx2267.png,[15]
1,pelvicfx3459.png,[15]


In [7]:
# preparing dataframe from 'Wrist' directory
wrist_images = os.listdir(WRIST_IMAGES)
wrist_images = [image_name for image_name in wrist_images if image_name.lower().endswith('.png')]
wrist_target = [[int(21)] for image_name in wrist_images]

wrist_df = pd.DataFrame({'Instance': wrist_images, 'Target': wrist_target})
wrist_df.head(2)

,Instance,Target
0,0123_0352338282_01_WRI-R2_M004.png,[21]
1,0842_1090707732_01_WRI-R3_M008.png,[21]


In [8]:
# merge all image df
df_all = pd.concat([mixed_df, pelvis_df, wrist_df, chest_df])
df_all.shape

(6400, 2)

In [9]:
def labels(label):
    if label == 0:
        return 'Abdomen'
    elif label == 1:
        return 'Ankle'
    elif label == 2:
        return 'Cervical Spine'
    elif label == 3:
        return 'Chest'
    elif label == 4:
        return 'Clavicles'
    elif label == 5:
        return 'Elbow'
    elif label == 6:
        return 'Feet'
    elif label == 7:
        return 'Finger'
    elif label == 8:
        return 'Forearm'
    elif label == 9:
        return 'Hand'
    elif label == 10:
        return 'Hip'
    elif label == 11:
        return 'Knee'
    elif label == 12:
        return 'Lower Leg'
    elif label == 13:
        return 'Lumbar Spine'
    elif label == 14:
        return 'Others'
    elif label == 15:
        return 'Pelvis'
    elif label == 16:
        return 'Shoulder'
    elif label == 17:
        return 'Sinus'
    elif label == 18:
        return 'Skull'
    elif label == 19:
        return 'Thigh'
    elif label == 20:
        return 'Thoracic Spine'
    elif label == 21:
        return 'Wrist'

In [10]:
# for multiclassfication problem using MultiLabelBinarizer()
mlb = MultiLabelBinarizer()

# transform labels into multilabelbinary format
encoded_labels = mlb.fit_transform(df_all['Target'].apply(lambda x : [int(label) for label in x]))
encoded_labels_df = pd.DataFrame(encoded_labels, columns=mlb.classes_)
encoded_labels_df.head(2)

,0,1,2,3,4,5,6,7,8,9,...,12,13,14,15,16,17,18,19,20,21
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0


In [11]:
encoded_labels.shape

(6400, 22)

In [12]:
# reset index and concat the dataframe with Instance and its MultiLabelBinaries
df_all.reset_index(drop=True, inplace=True)
encoded_labels_df.reset_index(drop=True, inplace=True)

df = pd.concat([df_all, encoded_labels_df], axis=1)
df.head(5)

,Instance,Target,0,1,2,3,4,5,6,7,...,12,13,14,15,16,17,18,19,20,21
0,1.2.826.0.1.3680043.8.498.10025629581362719970...,[0],1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1.2.826.0.1.3680043.8.498.10036150326276641158...,[15],0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
2,1.2.826.0.1.3680043.8.498.10038426859954986240...,[12],0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
3,1.2.826.0.1.3680043.8.498.10050991192143676483...,[14],0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
4,1.2.826.0.1.3680043.8.498.10053309524595490852...,[3],0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [13]:
for idx, column in enumerate(df.iloc[:,2:].columns, start=0):
    print(idx, labels(idx), df.iloc[:,2+idx:2+idx+1].sum().sum())

0 Abdomen 92
1 Ankle 54
2 Cervical Spine 81
3 Chest 2079
4 Clavicles 9
5 Elbow 25
6 Feet 78
7 Finger 17
8 Forearm 15
9 Hand 74
10 Hip 25
11 Knee 110
12 Lower Leg 26
13 Lumbar Spine 87
14 Others 120
15 Pelvis 221
16 Shoulder 41
17 Sinus 23
18 Skull 10
19 Thigh 15
20 Thoracic Spine 64
21 Wrist 3276


Images have different sizes we need to standardize them to (128, 128) pixels for this we will use cv2.resize()

In [14]:
X_images = []
Y_labels = []

In [15]:

# image preprocessing
for i, val in enumerate(df.Instance):
    mixed_images = [image for image in os.listdir(MIXED_IMAGES) if image.lower().endswith(('.png', 'jpeg'))]
    pelvis_images = [image for image in os.listdir(PELVIS_IMAGES) if image.lower().endswith(('.png', 'jpeg'))]
    wrist_images = [image for image in os.listdir(WRIST_IMAGES) if image.lower().endswith(('.png', 'jpeg'))]
    chest_images = [image for image in os.listdir(CHEST_IMAGES) if image.lower().endswith(('.png', 'jpeg'))]
    hand_images = [image for image in os.listdir(HAND_IMAGES) if image.lower().endswith(('.png', 'jpeg'))]

    if val in mixed_images:
        image_path = os.path.join(MIXED_IMAGES, val)
    elif val in pelvis_images:
        image_path = os.path.join(PELVIS_IMAGES, val)
    elif val in wrist_images:
        image_path = os.path.join(WRIST_IMAGES, val)
    elif val in chest_images:
        image_path = os.path.join(CHEST_IMAGES, val)
    elif val in hand_images:
        image_path = os.path.join(HAND_IMAGES, val)
    
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    img_resize = cv2.resize(img, dsize=(128, 128))
    img_norm = cv2.normalize(img_resize, None, alpha=0, beta=500, norm_type=cv2.NORM_MINMAX)
    # var 'X' will contain the array representation of orginal image and we add a new dimention to the array at start. For CNN we need the data in this shape batch_size, height, width, channels)
    X_images.append(np.expand_dims(img_norm, axis=-1))
    # flatten img_norm into 1d array and store it in list var 'Y'
    #Y_labels.append(np.ndarray.flatten(np.array(new_train_df.loc[i][new_train_df.columns[65:]])))

X_images = np.array(X_images)
Y_labels = np.array(encoded_labels)

In [16]:
x_images_train, x_images_test, y_labels_train, y_labels_test = train_test_split(X_images, Y_labels, test_size=0.15)

In [17]:
x_images_train.shape

(5440, 128, 128, 1)

In [18]:
y_labels_train.shape

(5440, 22)

In [19]:
# Load your data and preprocess it, including converting grayscale images to RGB (for ResNet Layer)
x_images_train_rgb = np.repeat(x_images_train, 3, axis=-1)
x_images_test_rgb = np.repeat(x_images_test, 3, axis=-1)

# Define the input shape
x_images_train_rgb[0].shape  # Use the shape of one image



(128, 128, 3)

In [20]:
print(tf.config.list_physical_devices('GPU'))

[]


CNN MODEL WITH MULTI-LAYERS
Using Convolutional Neural Network (CNN) and Keras API to desgin an image classification neutral network.

- 'Sequential' = is used to create a linear stack of layers
        model = tf.keras.models.Sequential()
        Sequential is the fundamental archetecture of neural networks. Think it as pipeline where data flows sequentially through each layer in the model. The output of one layer will become the input of the next layer. 

- 'InputLayer' = is defined to specify the input shape (128,128,1) 128x128 pixels and 1 channel for grayscale
        model.add(InputLayer(input_shape=(128,128,1)))
        While initialling the neural network especially Sequential model you should define the input shape of your data because:
        - This will initalize the weights and biases of the sebsequent layers correctly. This is needed so the network setup connections between the layers and allocate appropiate number of parameters for each layer
        - Specifing the input shape helps catch errors and inconsistancies earlier in the building process. If you try to connect a layer with the wrong input shape the framework will raise an error making it easier to debug and troubleshoot your model.

- 'Conv2D' = (CNN) layer is added for feature extraction
        model.add(Conv2D(64, kernal_size=(3,3), activation='relu', kernal_initializer='he_normal', padding='same'))
        Conv2D(CNN) parameters include 
        - '64' filters (kernals/channels): means in CNN each filter learns to detect different features in the input data 
        - 'kernal_size: this defines the filter size that slides over the input data meaning the (3x3) pixel filter will slide over the image to detect features
        - 'activation': this function help the model by allowing it to generate a non-linear output for each element in the image array. 'relu' (Rectified Linear Activation) allows non-linearity while being computationally efficent.
        - 'kenal_initialization': this defines the weight initialization techniques for filter weights. 'he_normal' is an initailization method that helps prevent vanishing gradients during training.
        - 'padding': this defines the padding strategy for input data before CNN operation. 'same' padding means that the input data will be padded with zeros in such a way that the output has the same spatial dimentions as the input. padding helps maintain the spatial dimensions of the input and output volumes after convolution.  

- 'MaxPooling2D' = (max pooling) layer is added to downsize the spatial dimensions to reduce computations
        model.add(MaxPooling2D(pool_size=(2,2)))
        'MaxPooling2D' is used in CNN to downsize the spatial dimentions (width, height) of the input data while retaining important information. This is done by retaining the maximum value from a group of neighbouring pixels in the input hence reducing the dimention of the data. '(2x2)' means that this operation takes a maximum value within each 2x2 window

- 'Dropout' = layer is added for regularization, which helps prevent overfitting by randomly setting fraction inputs to 0 during each update
        model.add(Dropout(0.6))
        Dropout layer is used to prevent overfitting in a neural network. Overfitting occurs when a model learn extremely well on the trained data but fails to generalize to new unseen data. Dropout helps improve model generalization by introducing randomness during training. At each training layer a different set of neurons is dropped out which means they are not used to compute the output of the layer. The dropout rate is the percentage of neurons that are dropped out. In this example 60% neurons in EACH layer will be randomly dropped out during training. The remaining 40% neurons will be used to compute the output of the layer.

- 'Flatten' = layer flattens the output from Conv2D into a vector (1-D vector) so that it can be processed by Dense Layers (fully connected layers)
        model.add(Flatten())
        Flatten layer reshapes the input data to one-dimentional array. This is placed at the end of the sequence of CNN layers just before the fully-connected layer Dense() in a neural network. The purpose is to flatten the input that can be fed into Dense() layer. This conversion is required because Dense() layers expect one-dimentional array as they are fully connected to establish network. For example if the CNN produces a feature map (batch_size, height, width, channels), the flatten size is the product of (height x weigth x channel) 

- 'Dense' = represent fully connected layers. These layers process the flattened features to make decisions. Layer as many units as classes in your classification problem (41 in this case). This uses the activation fucntion to output class probabilities 
        model.add(Dense(2048, activation='relu'))
        model.add(Dense(41, activation='softmax'))
        Dense layers are fully-connected layers they take input from CNN earlier layers extracted features to learn complex relationships in the data. Dense() takes in number of neurons(units) as a parameter in the Dense() layer. Each neuron will learn to capture differnet patterns and features from the input data to final class scores prediction in a classification problem. 
        - activation function 'relu' (Rectification Linear Activation) applies to the output of the neurons in the Dense() layer. ReLU is a popular choice for acitvation function because its non-linear while being computationally efficent.
        - activation function 'softmax' is usually used in the output layer of the neural networks, especially for multi-class classifications problems. The output of the softmax function defines the probalities that the input belongs to each class

- 'model.compile' = the model is compiled using 'Adam' optimizer assuming binary (0 to 1) classification and accuracy as the evaluation metric
        model.compile(optimizer='Adam', loss='BinaryCrossentropy', metrics=['accuracy'])
        During model training compile is an essential function that optimizes the model performace. It basically drives a computational graph that defines how the forward and backward passes will be executed during training. It sets a stage for optimization process by specifying how the model will learn from the data and udpate its parameters to make better predictions. The graph is also crutial for CPU/GPU utilization and automatic differenciation between predictions.
        - loss function: is a mathematical function the qunatifies the difference between predicted and actual values in a model. Its measures the performace and guides the optimization process by providing feedback on how well it fits the data. 'BinaryCrossentropy' is a specific type of loss function used when dealing with binaey classification problems (where the output is a binary value like 0 to 1). It measures the difference between the predicted probabilities and actual binary labels.
        - optimizer: is responsible for updating model weights (weight are connection management between two basic units within a neural network. In CNN these are the kernal/filters i.e the matrices that you use to perform convolution in a layer) to minimze the loss function. 'Adam' (Adaptive Movement Estimation) is an optimizer in deeplearning that dynamically adjusts the learning rate during training, which helps improve training speed.
        - metrics: is used to monitor model performance during training. The 'accuracy' metric measures the propotion of corrected predicted instances out of the total instances in the dataset. It is a common metric for classification tasks.
- 'summary' = print the summary of model architecture. This displays the output shape of each layer and the number of trainable parameters
        model.summary()
        During training keras provides us the summary() function to overview the archetecture of the neural network. It displays the model layers, their output shapes and number of trainable parameters in each layer. This information is crutial to understand the structure and complexity of the model.

While adding additional CNN layers you will notice a linear increase in filter numbers. This is a common CNN archetecture choice known as pyramid. This model design strategy has been found effective for the follwing reason:
- Hierarcial Feature Learning: As we move deeper into the network we increase the filters hence allowing the network to learn features at different levels of abstraction, just like a pyramid. Small filters capture global features whereas larger filters capture more finer details

In [21]:
# Create the ResNet50 base model (Transfer learning - using an exiting trained model)
# include_top is False because we dont want ResNet classfiying images to 1000 differenct classes. We will add our own classes
# weights is imagenet is telling the layer to load weights that were trained on the ImageNet dataset. This will help our model to learn faster.
base_model = ResNet50(input_shape=(128,128,3), include_top=False, weights='imagenet')

# Iterate over all layers and freeze the layers weights so they will not be updated during trainine
for layer in base_model.layers:
    layer.trainable = False

# Create your custom model
model = tf.keras.models.Sequential()

# Add the ResNet50 base model
model.add(base_model)

# Add your existing layers
model.add(Conv2D(512, kernel_size=(3,3), activation='relu', kernel_initializer='he_normal', padding='same'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Dropout(0.50))
model.add(Flatten())

model.add(Dense(4096, activation='relu'))
model.add(Dense(2048, activation='relu'))
model.add(Dense(22, activation='sigmoid'))

# Compile the model
model.compile(optimizer='Adam', loss='BinaryCrossentropy', metrics=['accuracy'])
model.summary()


2023-08-19 14:45:39.593121: E itex/core/wrapper/itex_gpu_wrapper.cc:49] Could not load Intel Extension for Tensorflow* GPU backend, GPU will not be used.
If you need help, create an issue at https://github.com/intel/intel-extension-for-tensorflow/issues
2023-08-19 14:45:39.594539: E itex/core/wrapper/itex_gpu_wrapper.cc:49] Could not load Intel Extension for Tensorflow* GPU backend, GPU will not be used.
If you need help, create an issue at https://github.com/intel/intel-extension-for-tensorflow/issues


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 4, 4, 2048)        23587712  
                                                                 
 conv2d (Conv2D)             (None, 4, 4, 512)         9437696   
                                                                 
 max_pooling2d (MaxPooling2  (None, 2, 2, 512)         0         
 D)                                                              
                                                                 
 dropout (Dropout)           (None, 2, 2, 512)         0         
                                                                 
 flatten (Flatten)           (None, 2048)              0         
                                                                 
 dense (Dense)               (None, 4096)              8392704   
                                                        

2023-08-19 14:45:41.389137: E itex/core/wrapper/itex_gpu_wrapper.cc:49] Could not load Intel Extension for Tensorflow* GPU backend, GPU will not be used.
If you need help, create an issue at https://github.com/intel/intel-extension-for-tensorflow/issues


In [22]:
model.fit(
     x_images_train_rgb,
     y_labels_train,
     batch_size=5,
     callbacks=[
         tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=5),
         tf.keras.callbacks.TensorBoard(log_dir=os.path.join(os.getcwd(), 'logs'), histogram_freq=1)
     ],
     epochs=1
 )

2023-08-19 14:45:42.009361: E itex/core/wrapper/itex_gpu_wrapper.cc:49] Could not load Intel Extension for Tensorflow* GPU backend, GPU will not be used.
If you need help, create an issue at https://github.com/intel/intel-extension-for-tensorflow/issues
2023-08-19 14:45:42.082529: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type CPU is enabled.
2023-08-19 14:45:42.091010: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type CPU is enabled.
2023-08-19 14:45:42.099511: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type CPU is enabled.
2023-08-19 14:45:43.986936: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type CPU is enabled.


1088/1088 [==============================] - 448s 409ms/step - loss: 0.0779 - accuracy: 0.8250


In [23]:
results = model.evaluate(x_images_test_rgb, y_labels_test)

print('loss: ', results[0])
print('accuracy: ', results[1])

2023-08-19 14:53:09.772628: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type CPU is enabled.
2023-08-19 14:53:09.776407: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type CPU is enabled.
2023-08-19 14:53:09.779934: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type CPU is enabled.
2023-08-19 14:53:10.231535: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type CPU is enabled.


30/30 [==============================] - 14s 449ms/step - loss: 0.0450 - accuracy: 0.8604
loss:  0.04504675790667534
accuracy:  0.8604166507720947


In [24]:
model.save('body_part_classification.h5')

/home/zaeem/Desktop/projects/jupyter_notebooks/body_part_classification/.venv/lib/python3.8/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [29]:
loaded_model = tf.keras.models.load_model('keras_model.h5')

In [30]:
# Load and preprocess the image
wrist_image_path = WRIST_IMAGES
images_list = os.listdir(wrist_image_path)
image_files = [image for image in images_list if image.lower().endswith(('.png'))][200:210]
wrist_image_labels = []

for image_file in image_files:
    img_path = os.path.join(wrist_image_path, image_file)
    img = plt.imread(img_path)

    # Convert grayscale to RGB
    if img.ndim == 2:
        img = np.repeat(img[..., np.newaxis], 3, axis=-1) 
    
    # Resize image input for the model and expand dimensions to create a batch of size 1
    img_resized = tf.image.resize(img, (128, 128))
    img = tf.expand_dims(img_resized, axis=0) 

    # Make predictions using the loaded model
    prediction = loaded_model.predict(img)
    predicted_class = np.argmax(prediction)
    print(prediction)
    print('Prediction class: ', predicted_class)
    print('Prediction label: ', labels(int(predicted_class)))
    
    wrist_image_labels.append(labels(int(predicted_class)))

total_images = len(wrist_image_labels)
correctly_predicted = len([labels for labels in wrist_image_labels if 'Wrist' in labels])
wrong_predicted_labels = [labels for labels in wrist_image_labels if 'Wrist' not in labels]
accuracy_percentage = (correctly_predicted / total_images) * 100

print('Total wrist images predicted:', total_images)
print('Total wrist images correctly predicted:', correctly_predicted)
print('Wrong predicted labels: ', wrong_predicted_labels)
print('Percentage accuracy:', accuracy_percentage)

2023-08-19 15:41:10.537548: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type CPU is enabled.
2023-08-19 15:41:10.544523: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type CPU is enabled.
2023-08-19 15:41:10.549046: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type CPU is enabled.


ValueError: in user code:

    File "/home/zaeem/Desktop/projects/jupyter_notebooks/body_part_classification/.venv/lib/python3.8/site-packages/keras/src/engine/training.py", line 2341, in predict_function  *
        return step_function(self, iterator)
    File "/home/zaeem/Desktop/projects/jupyter_notebooks/body_part_classification/.venv/lib/python3.8/site-packages/keras/src/engine/training.py", line 2327, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/home/zaeem/Desktop/projects/jupyter_notebooks/body_part_classification/.venv/lib/python3.8/site-packages/keras/src/engine/training.py", line 2315, in run_step  **
        outputs = model.predict_step(data)
    File "/home/zaeem/Desktop/projects/jupyter_notebooks/body_part_classification/.venv/lib/python3.8/site-packages/keras/src/engine/training.py", line 2283, in predict_step
        return self(x, training=False)
    File "/home/zaeem/Desktop/projects/jupyter_notebooks/body_part

In [27]:
!tensorboard --logdir ./logs

2023-08-19 14:53:30.207899: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-08-19 14:53:30.210507: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-19 14:53:30.243401: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-19 14:53:30.243778: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-19 14:53:30.965773: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

In [28]:
img_path = os.path.join(DATA_DIR, 'happy.png')
img = cv2.imread(img_path)
img_resized = tf.image.resize(img, (128,128))
img_resized = img_resized / 255.0
plt.imshow(img_resized)
img = tf.expand_dims(img_resized, axis=0)
prediction = loaded_model.predict(img)
print(labels(int(np.argmax(prediction))))

[ WARN:0@716.393] global loadsave.cpp:248 findDecoder imread_('/home/zaeem/Desktop/projects/jupyter_notebooks/body_part_classification/data/happy.png'): can't open/read file: check file path/integrity


ValueError: Attempt to convert a value (None) with an unsupported type (<class 'NoneType'>) to a Tensor.